In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import tree  
from sklearn.model_selection import RandomizedSearchCV
#import pydotplus #pip install pydotplus
import collections
from IPython.display import display, HTML, Image

In [2]:
#from google.colab import files


#uploaded = files.upload()


In [3]:
import io
#dataFrame= pd.read_csv(io.BytesIO(uploaded['test.csv']))
dataFrame=pd.read_csv (r'C:\Users\ritis\Documents\test.csv')

In [4]:
dataFrame.head(5)

,x1,x2,x3,class
0,132,1342,7018,1
1,312,5197,8910,1
2,213,1927,6019,0
3,451,2719,7109,0
4,345,4860,7180,0


In [5]:
print(dataFrame.shape)
dataFrame.dtypes

(20, 4)


x1       int64
x2       int64
x3       int64
class    int64
dtype: object

In [6]:
dataFrame.describe(include = 'all').T

,count,mean,std,min,25%,50%,75%,max
x1,20.0,477.60,235.592512,125.0,336.75,469.5,599.75,920.0
x2,20.0,2922.60,1457.304664,1309.0,1780.50,2463.0,4264.75,5402.0
x3,20.0,7749.40,1305.852027,6019.0,6961.00,7189.0,8925.25,9999.0
class,20.0,0.45,0.510418,0.0,0.00,0.0,1.00,1.0


In [7]:
# Correlation logic will be added here

"""
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


label_encoder = LabelEncoder()
newDF.iloc[:,0] = label_encoder.fit_transform(newDF.iloc[:,0]).astype('float64')"""

dataFrame.corr()

,x1,x2,x3,class
x1,1.000000,0.050558,-0.157668,0.123251
x2,0.050558,1.000000,0.076676,0.052969
x3,-0.157668,0.076676,1.000000,-0.445481
class,0.123251,0.052969,-0.445481,1.000000


In [8]:
# splitting the data first

df_train, df_test = train_test_split(dataFrame, test_size= 0.5, random_state =1)
print("Rows and columns for Train:",df_train.shape)
print("Rows and columns for Test",df_test.shape)

Rows and columns for Train: (10, 4)
Rows and columns for Test (10, 4)


In [9]:
# preparing the model

# features to consider for model building
features = dataFrame.columns
features = dataFrame.drop(["class"], axis=1).columns  
#replace class with target in the given dataset

In [10]:
# Grid Search the data

clf = RandomForestClassifier(n_estimators=100 , oob_score = True, n_jobs = -1,random_state =50, max_features = None)

# Create the parameter grid based on the results of random search 
param_distributions = {'bootstrap': [True], 'max_depth': range(10,100,20), 'min_samples_leaf': [1, 2, 4], 'n_estimators': range(100,1000,200)}


# Instantiate the grid search model
random_search = RandomizedSearchCV(estimator = clf, param_distributions = param_distributions, 
                          cv = 3, n_jobs = -1, verbose = 2,n_iter=5,random_state=42)
print(param_distributions)
random_search.fit(df_train[features], df_train["class"])
print("The best is" ,random_search.best_params_)
best_clf = random_search.best_estimator_

{'bootstrap': [True], 'max_depth': range(10, 100, 20), 'min_samples_leaf': [1, 2, 4], 'n_estimators': range(100, 1000, 200)}
Fitting 3 folds for each of 5 candidates, totalling 15 fits
The best is {'n_estimators': 900, 'min_samples_leaf': 1, 'max_depth': 10, 'bootstrap': True}


In [11]:
# Initialise Random Forest
# set the parameters to the best fit

#newRF = RandomForestClassifier(n_estimators=100 , oob_score = True, n_jobs = -1,random_state =50, max_features = None,bootstrap= True,min_samples_leaf=1,max_depth=10)
# Fit on training data
best_clf.fit(df_train[features], df_train["class"])
# Make predictions
predictions = best_clf.predict(df_test[features])
probs = best_clf.predict_proba(df_test[features])
# Performance measures
score = best_clf.score(df_test[features], df_test["class"])
print("Accuracy of prediction is: ", score)
print(confusion_matrix(df_test["class"],predictions))
print(classification_report(df_test["class"],predictions))

Accuracy of prediction is:  0.4
[[2 4]
 [2 2]]
              precision    recall  f1-score   support

           0       0.50      0.33      0.40         6
           1       0.33      0.50      0.40         4

    accuracy                           0.40        10
   macro avg       0.42      0.42      0.40        10
weighted avg       0.43      0.40      0.40        10



In [12]:
import joblib
from sklearn.ensemble import RandomForestClassifier
# create RF
rf = RandomForestClassifier()
# fit on some data
rf.fit(df_train,df_test)

# save
joblib_file = "my_random_forest.pkl"
joblib.dump(rf,joblib_file)
import os
os.getcwd()
print("Model dumped!")

# load
loaded_rf = joblib.load(joblib_file)

a = [[300,1200,9810,1]]
b =loaded_rf.predict(a)
print("Predicted=%s" % (b))

import os
os.getcwd()

Model dumped!
Predicted=[[ 506 1378 8710    0]]


'C:\\Users\\ritis\\Downloads'

In [ ]:
from flask import Flask, request, jsonify
#from sklearn.externals import joblib
import traceback
import pandas as pd
import numpy as np

# Your API definition
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    if lr:
        try:
            json_ = request.json
            print(json_)
            query = pd.get_dummies(pd.DataFrame(json_))
            query = query.reindex(columns=model_columns, fill_value=0)

            prediction = list(lr.predict(query))

            return jsonify({'prediction': str(prediction)})

        except:

            return jsonify({'trace': traceback.format_exc()})
    else:
        print ('Train the model first')
        return ('No model here to use')

if __name__ == '__main__':
    try:
        port = int(sys.argv[1]) # This is for a command-line input
    except:
        port = 5000 # If you don't provide any port the port will be set to 12345

    lr = joblib.load("my_random_forest.pkl") # Load "model.pkl"
    print ('Model loaded')
    #app.run(port=port, debug=True)
    print("check")
    app.run(port=port,debug=True, use_reloader=False)
    print("check2")

Model loaded
check
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Jul/2021 13:37:32] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [03/Jul/2021 13:37:32] "GET /favicon.ico HTTP/1.1" 404 -
